In [108]:
import datetime
import importlib.util
import sys
from tradingstrategy.client import Client
from tradeexecutor.utils.default_strategies import get_default_strategies_path
from tradeexecutor.strategy.execution_context import notebook_execution_context


from tradeexecutor.utils.default_strategies import get_default_strategies_path
from tradeexecutor.strategy.strategy_module import read_strategy_module

strategies_folder = get_default_strategies_path()
strategy_path = strategies_folder / "spot-and-short-momentum-tuned.py"

strategy_mod = read_strategy_module(strategy_path)

print(f"Backtesting for strategy {strategy_path}, engine version {strategy_mod.trading_strategy_engine_version}")

client = Client.create_jupyter_client()

# contains start_at, end_at backtesting range
universe_options = strategy_mod.get_universe_options()

strategy_universe = strategy_mod.create_trading_universe(
    datetime.datetime.utcnow(),
    client,
    notebook_execution_context,
    universe_options,
)

data_universe = strategy_universe.data_universe

Backtesting for strategy /Users/moo/code/ts/trade-executor/strategies/spot-and-short-momentum-tuned.py, engine version 0.3
Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


## Explore momentum by weekday



In [109]:
import pandas as pd
import numpy as np


def calculate_momentum_by_weekday(df, momentum_period: pd.Timedelta, weekday: int) -> pd.Series:
    # Monday = 0 and Sunday = 6
    index = []
    values = []
    total_days = 0
    matches = df.index[df.index.weekday==weekday]
    for date in matches:       
        first_ts = date + momentum_period
        total_days += 1
        if first_ts not in df.index:
            print("Skipping", first_ts)
            continue 
        first = df["close"][first_ts]
        last = df["open"][date]
        momentum = (last - first) / first 

        assert momentum != np.NaN, f"Bad momentum: {momentum}, {first_ts} - {date}, {first}->{last}"
        index.append(date)
        values.append(momentum)

    assert len(values) > 1, f"Could not calculate: {momentum_period} for weekday {weekday}, over {total_days} days, index is {len(df.index)} entries, we have {len(matches)} matches"
    
    return pd.Series(values, index)




In [110]:
# Do ETH first
import calendar
from tradingstrategy.chain import ChainId
import plotly.express as px

pair = data_universe.pairs.get_pair_by_human_description((ChainId.polygon, "uniswap-v3", "WETH", "USDC"))
df = data_universe.candles.get_candles_by_pair(pair)
assert df is not None
# momentums = pd.DataFrame()

series_by_day = {}

# display(df)

for day in range(0, 7):
    day_name = calendar.day_name[day]
    print(f"Creating for {day_name}")    
    series = calculate_momentum_by_weekday(df, pd.Timedelta(days=-7), day)
    series_by_day[day_name] = series
    # display(momentums[day_name])

    # fig = px.histogram(series)
    # fig.show(renderer="svg")

# momentums.index = df.index
momentums = pd.DataFrame(series_by_day)

# display(momentums)
# fig = px.histogram(momentums["Tuesday"])
# fig.show(renderer="svg")
# display(momentums)


fig = px.histogram(momentums, marginal="violin", barmode="group")
fig.show()

Creating for Monday
Skipping 2022-07-04 00:00:00
Creating for Tuesday
Skipping 2022-07-05 00:00:00
Creating for Wednesday
Skipping 2022-07-06 00:00:00
Creating for Thursday
Skipping 2022-06-30 00:00:00
Creating for Friday
Skipping 2022-07-01 00:00:00
Creating for Saturday
Skipping 2022-07-02 00:00:00
Creating for Sunday
Skipping 2022-07-03 00:00:00


In [111]:
def calculate_profit_chances(df, profit_period: pd.Timedelta, weekday: int, threshold) -> pd.Series:
    # Monday = 0 and Sunday = 6
    index = []
    values = []
    total_days = 0
    assert threshold > 1
    matches = df.index[df.index.weekday==weekday]
    for date in matches:               
        first_ts = date 
        last_ts = date + profit_period

        start_price = df["open"][first_ts]
        threshold_price = start_price * threshold

        closes = df["close"][first_ts + pd.Timedelta(seconds=1):last_ts]
        profit_hit = closes.max() > threshold_price
        index.append(date)
        values.append(profit_hit)

    return pd.Series(values, index)
     

for day in range(0, 7):
    day_name = calendar.day_name[day]
    series = calculate_profit_chances(df, pd.Timedelta(days=7), day, threshold=1.001)
    series_by_day[day_name] = series
    # display(momentums[day_name])

profit_periods = pd.DataFrame(series_by_day)

for day in range(0, 7):
    day_name = calendar.day_name[day]
    period = profit_periods[day_name]
    print(f"Profitable trading periods start by {day_name}: {period.sum()}")


Creating for Monday
Creating for Tuesday
Creating for Wednesday
Creating for Thursday
Creating for Friday
Creating for Saturday
Creating for Sunday
Profitable trading periods start by Monday: 50
Profitable trading periods start by Tuesday: 54
Profitable trading periods start by Wednesday: 51
Profitable trading periods start by Thursday: 56
Profitable trading periods start by Friday: 54
Profitable trading periods start by Saturday: 52
Profitable trading periods start by Sunday: 54


In [112]:
# weekday -> period -> pair id

# periods = [6, 7, 8]
# pair_dfs = defaultdict(lambda: defaultdict(int))

# data_df = {}
# for pair in data_universe.pairs.get_all_pair_ids():
#     data_df[pair] = data_universe.candles.get_candles_by_pair(pair)

# for pair in data_universe.pairs.get_all_pair_ids():
#     data = data_df[pair]
#     for day in range(0, 7):
#         for period in periods:
#             pair_dfs[day][period][pair] = calculate_momentum_by_weekday(period, day)
